In [ ]:
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

In [ ]:
#all variables
data_classified = 'Data/classified'
data_raw = 'Data/raw'
data_alpha = 'Data/diversity'

In [ ]:
#General outline; need to wait before differences between groups have been discovered in beta diversity

In [ ]:
#! qiime feature-table filter-features \
  #  --i-table $data_dir/dada2_table.qza \
  #  --p-min-frequency 25 \
  #  --p-min-samples 4 \
  #  --o-filtered-table $data_dir/table_abund.qza

In [ ]:
#e.g. with the env 
#! qiime feature-table filter-samples \
#    --i-table $data_dir/table_abund.qza \
#    --m-metadata-file $data_dir/metadata.tsv \
#    --p-where "[env]='gut' or [env]='oral'" \
#    --o-filtered-table $data_dir/table_abund_gut_oral.qza